<a href="https://colab.research.google.com/github/aino-capm/googlecolab/blob/main/0_docid%E3%81%AE%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ライブラリーのインポート
import datetime
import requests
import pandas as pd

In [ ]:
#対象日の指定
start_date = datetime.date(2021, 5,1)
end_date = datetime.date(2021, 7,31)

In [ ]:
#day_listの作成
period = end_date - start_date
period = int(period.days)
day_list = []
for d in range(period):
    day = start_date + datetime.timedelta(days=d)
    day_list.append(day)
    
day_list.append(end_date)

In [ ]:
#対象期間における提出データの取得
report_list =[]
for day in day_list:
    url = "https://disclosure.edinet-fsa.go.jp/api/v1/documents.json"
    params = {"date": day, "type": 2}
    res = requests.get(url, params=params)
    json_data = res.json()

    #以下のfor文は上のfor文の続きに書かないと、結局、最終日付分しか取得できないことになる。
    for num in range(len(json_data["results"])):
        ordinance_code= json_data["results"][num]["ordinanceCode"]
        form_code= json_data["results"][num]["formCode"]
        
        if ordinance_code == "010" and  form_code =="030000" :
            company_name=json_data["results"][num]["filerName"]
            edi={ '会社名':company_name,
                        '書類名':json_data["results"][num]["docDescription"],           
                        'docID':json_data["results"][num]["docID"],
                        '証券コード':json_data["results"][num]["secCode"],
                        'EDINETコード':json_data["results"][num]["edinetCode"],
                        '決算期':json_data["results"][num]["periodEnd"],
                        '提出日': day             }
            report_list.append(edi)


In [ ]:
%load_ext google.colab.data_table

In [ ]:
df = pd.DataFrame(report_list)
df

In [ ]:
df_0331= df[df["決算期"]=="2021-03-31"].sort_values("提出日")
df_0331

In [ ]:
df_info = pd.read_csv("/content/EdinetcodeDlInfo.csv",encoding="cp932",skiprows=[0])
df_info = df_info.rename(columns={"提出者名":"会社名"})
df_info =df_info[["会社名","提出者業種"]]
df_info 

In [ ]:
df_210331 = pd.merge(df_0331,df_info,how="inner",on="会社名")
df_210331.to_csv("/content/drive/MyDrive/0_kikagaku/NPL_APP/edinetcodelist_2103.csv")